# Read clean , import functions

In [ ]:
import pandas as pd
import numpy as np
from utils import create_mmsi_dict_from_file
from utils import filter_stationary_ships
from utils import prepare_training_data

In [ ]:
np.set_printoptions(precision=3, suppress=True)

In [ ]:
import pandas as pd

df_clean = pd.read_csv("data/ais_data_5min_clean.csv")
df_clean.head()

import numpy as np

def encode_cog(df):
    # Convert degrees → radians
    rad = np.deg2rad(df["COG"].values)

    df["COG_sin"] = np.sin(rad)
    df["COG_cos"] = np.cos(rad)

    return df.drop(columns=["COG"])

df_clean = encode_cog(df_clean)


In [ ]:
file_name = "data/mmsi_type.txt"
mmsi_map = create_mmsi_dict_from_file(file_name)


if mmsi_map:
    print("--- Successfully created dictionary ---")

# Create training data

In [ ]:
%matplotlib inline
# Import deep learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")


# Prepare data

In [ ]:
# Configuration for model training
SEQUENCE_LENGTH = 20
FEATURES = ["Latitude", "Longtitude", "SOG", "COG_sin", "COG_cos"] 
TARGET_FEATURES = ["Latitude", "Longtitude", "SOG", "COG_sin", "COG_cos"]
MIN_SEGMENT_LENGTH = SEQUENCE_LENGTH + 5
INTERVAL = 5

In [ ]:
#Prepare sequences
X, y, segment_info = prepare_training_data(
    df_clean,
    SEQUENCE_LENGTH,
    FEATURES,
    TARGET_FEATURES,
    MIN_SEGMENT_LENGTH
)

# Display some statistics
segment_lengths = [s['length'] for s in segment_info]

In [ ]:
# Split data by ships (MMSI)
# Get unique MMSIs from segment_info
unique_mmsis = list(set([seg['mmsi'] for seg in segment_info]))
n_ships = len(unique_mmsis)

# Split ships into train (64%), val (16%), test (20%)
mmsi_temp, mmsi_test = train_test_split(
    unique_mmsis, test_size=0.2, random_state=42, shuffle=True
)

mmsi_train, mmsi_val = train_test_split(
    mmsi_temp, test_size=0.2, random_state=42, shuffle=True
)

# Create sets of MMSIs for fast lookup
mmsi_train_set = set(mmsi_train)
mmsi_val_set = set(mmsi_val)
mmsi_test_set = set(mmsi_test)

# Split sequences based on which ship they belong to
train_indices = [i for i, seg in enumerate(segment_info) if seg['mmsi'] in mmsi_train_set]
val_indices = [i for i, seg in enumerate(segment_info) if seg['mmsi'] in mmsi_val_set]
test_indices = [i for i, seg in enumerate(segment_info) if seg['mmsi'] in mmsi_test_set]

# Get the actual sequences for each set (RAW)
X_train_raw = X[train_indices]
y_train_raw = y[train_indices]

X_val_raw = X[val_indices]
y_val_raw = y[val_indices]

X_test_raw = X[test_indices]
y_test_raw = y[test_indices]

In [ ]:
# Normalize the data
print("="*60)
print("Normalizing Data")
print("="*60)

# Reshape X_train for normalization
n_samples_train, n_timesteps, n_features = X_train_raw.shape
X_train_reshaped = X_train_raw.reshape(-1, n_features)

# Fit scaler on training data
scaler_X = StandardScaler()
X_train_normalized_reshaped = scaler_X.fit_transform(X_train_reshaped)
X_train = X_train_normalized_reshaped.reshape(n_samples_train, n_timesteps, n_features)

# Transform val and test
X_val_reshaped = X_val_raw.reshape(-1, n_features)
X_val = scaler_X.transform(X_val_reshaped).reshape(X_val_raw.shape[0], n_timesteps, n_features)

X_test_reshaped = X_test_raw.reshape(-1, n_features)
X_test = scaler_X.transform(X_test_reshaped).reshape(X_test_raw.shape[0], n_timesteps, n_features)

# Normalize targets
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train_raw)
y_val = scaler_y.transform(y_val_raw)
y_test = scaler_y.transform(y_test_raw)

print(f"Input data normalized")
print(f"Target data normalized")
print(f"\nFeature means: {scaler_X.mean_}")
print(f"Feature stds: {scaler_X.scale_}")
print(f"\nTarget means: {scaler_y.mean_}")
print(f"Target stds: {scaler_y.scale_}")

In [ ]:
# Create delta features from df_clean
print("="*60)
print("Creating Delta Features")
print("="*60)

def create_delta_features(df):
    """
    Create delta (relative change) features from absolute positions.
    
    For each segment, compute:
    - delta_lat = current_lat - previous_lat
    - delta_lon = current_lon - previous_lon
    - Keep SOG and COG as-is
    """
    results = []
    
    for (mmsi, seg), group in df.groupby(["MMSI", "Segment"]):
        group = group.sort_values("Timestamp").copy()
        
        # Calculate deltas
        group["delta_lat"] = group["Latitude"].diff()
        group["delta_lon"] = group["Longtitude"].diff()
        
        # First row will have NaN deltas, so we drop it
        group = group.dropna(subset=["delta_lat", "delta_lon"])
        
        results.append(group)
    
    return pd.concat(results, ignore_index=True)

# Create delta dataframe
df_delta = create_delta_features(df_clean)

print(f"Original data shape: {df_clean.shape}")
print(f"Delta data shape: {df_delta.shape}")
print(f"\nNew columns added: delta_lat, delta_lon")
print(f"\nDelta statistics:")
print(df_delta[["delta_lat", "delta_lon", "SOG", "COG_sin", "COG_cos"]].describe())

In [ ]:
# Configuration for Delta-Based GRU model with Absolute Position
DELTA_SEQUENCE_LENGTH = 10  # Use 10 minutes of delta history
# Add absolute position (Latitude, Longtitude) to help model know where it is
DELTA_INPUT_FEATURES = ["delta_lat", "delta_lon", "SOG", "COG_sin", "COG_cos", "Latitude", "Longtitude"]
DELTA_TARGET_FEATURES = ["delta_lat", "delta_lon", "SOG", "COG_sin", "COG_cos",]
MIN_DELTA_SEGMENT_LENGTH = DELTA_SEQUENCE_LENGTH + 5

print("="*60)
print("Delta Model Configuration (with Absolute Position)")
print("="*60)
print(f"Sequence length: {DELTA_SEQUENCE_LENGTH} minutes")
print(f"Input features: {DELTA_INPUT_FEATURES}")
print(f"  - Deltas: delta_lat, delta_lon (relative movement)")
print(f"  - Dynamics: SOG, COG (speed and heading)")
print(f"  - Position: Latitude, Longtitude (absolute location)")
print(f"Target features: {DELTA_TARGET_FEATURES}")
print(f"Minimum segment length: {MIN_DELTA_SEGMENT_LENGTH} minutes")
print("\n✓ Model will now know both WHERE it is and HOW it's moving")

In [ ]:
# Prepare delta-based training data
print("="*60)
print("Preparing Delta-Based Training Data")
print("="*60)

# Prepare sequences using delta features
X_delta, y_delta, delta_segment_info = prepare_training_data(
    df_delta, 
    DELTA_SEQUENCE_LENGTH, 
    DELTA_INPUT_FEATURES, 
    DELTA_TARGET_FEATURES, 
    MIN_DELTA_SEGMENT_LENGTH
)

print(f"Total delta sequences created: {len(X_delta)}")
print(f"Input shape: {X_delta.shape}")
print(f"Target shape: {y_delta.shape}")
print(f"Segments used: {len(delta_segment_info)}")
print(f"Average sequences per segment: {len(X_delta) / len(delta_segment_info):.1f}")

# Display statistics
segment_lengths_delta = [s['length'] for s in delta_segment_info]
print(f"\nSegment statistics:")
print(f"  Min length: {min(segment_lengths_delta)} minutes")
print(f"  Max length: {max(segment_lengths_delta)} minutes")
print(f"  Mean length: {np.mean(segment_lengths_delta):.1f} minutes")
print(f"  Median length: {np.median(segment_lengths_delta):.1f} minutes")

In [ ]:
# Normalize delta data
print("="*60)
print("Normalizing Delta Data")
print("="*60)

# Reshape X for normalization
n_samples_delta, n_timesteps_delta, n_features_delta = X_delta.shape
X_delta_reshaped = X_delta.reshape(-1, n_features_delta)

# Fit scaler on delta training data
scaler_X_delta = StandardScaler()
X_delta_normalized = scaler_X_delta.fit_transform(X_delta_reshaped)
X_delta_normalized = X_delta_normalized.reshape(n_samples_delta, n_timesteps_delta, n_features_delta)

# Normalize delta targets
scaler_y_delta = StandardScaler()
y_delta_normalized = scaler_y_delta.fit_transform(y_delta)

print(f"Input data normalized: {X_delta_normalized.shape}")
print(f"Target data normalized: {y_delta_normalized.shape}")
print(f"\nFeature means: {scaler_X_delta.mean_}")
print(f"Feature stds: {scaler_X_delta.scale_}")

In [ ]:
# Split delta data by ships (MMSI)
print("="*60)
print("Splitting Delta Data by Ships (MMSI)")
print("="*60)

# Get unique MMSIs from delta segment_info
unique_mmsis_delta = list(set([seg['mmsi'] for seg in delta_segment_info]))
n_ships_delta = len(unique_mmsis_delta)

print(f"Total unique ships: {n_ships_delta}")

# Split ships: 64% train, 16% val, 20% test
mmsi_temp_delta, mmsi_test_delta = train_test_split(
    unique_mmsis_delta, test_size=0.2, random_state=42, shuffle=True
)

mmsi_train_delta, mmsi_val_delta = train_test_split(
    mmsi_temp_delta, test_size=0.2, random_state=42, shuffle=True
)

print(f"\nShips in training set: {len(mmsi_train_delta)} ({len(mmsi_train_delta)/n_ships_delta*100:.1f}%)")
print(f"Ships in validation set: {len(mmsi_val_delta)} ({len(mmsi_val_delta)/n_ships_delta*100:.1f}%)")
print(f"Ships in test set: {len(mmsi_test_delta)} ({len(mmsi_test_delta)/n_ships_delta*100:.1f}%)")

# Create sets for fast lookup
mmsi_train_set_delta = set(mmsi_train_delta)
mmsi_val_set_delta = set(mmsi_val_delta)
mmsi_test_set_delta = set(mmsi_test_delta)

# Split sequences based on ship MMSI
train_indices_delta = [i for i, seg in enumerate(delta_segment_info) if seg['mmsi'] in mmsi_train_set_delta]
val_indices_delta = [i for i, seg in enumerate(delta_segment_info) if seg['mmsi'] in mmsi_val_set_delta]
test_indices_delta = [i for i, seg in enumerate(delta_segment_info) if seg['mmsi'] in mmsi_test_set_delta]

# Get actual sequences
X_train_delta = X_delta_normalized[train_indices_delta]
y_train_delta = y_delta_normalized[train_indices_delta]

X_val_delta = X_delta_normalized[val_indices_delta]
y_val_delta = y_delta_normalized[val_indices_delta]

X_test_delta = X_delta_normalized[test_indices_delta]
y_test_delta = y_delta_normalized[test_indices_delta]

print(f"\nSequences in training set: {X_train_delta.shape[0]} ({X_train_delta.shape[0]/X_delta_normalized.shape[0]*100:.1f}%)")
print(f"Sequences in validation set: {X_val_delta.shape[0]} ({X_val_delta.shape[0]/X_delta_normalized.shape[0]*100:.1f}%)")
print(f"Sequences in test set: {X_test_delta.shape[0]} ({X_test_delta.shape[0]/X_delta_normalized.shape[0]*100:.1f}%)")

print("\n✓ Delta data split by ships - no temporal leakage between sets!")

# Create model

In [ ]:
""" from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def build_rnn_model(input_shape, output_dim):
    model = Sequential()

    # Stacked RNN layers
    model.add(SimpleRNN(2*128, return_sequences=True, activation='tanh', input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(SimpleRNN(2*64, return_sequences=False, activation='tanh'))

    # Dense layers for nonlinear mapping
    model.add(BatchNormalization())
    model.add(Dense(2*128, activation='relu'))
    model.add(Dropout(0.3))

    # Final regression output
    model.add(Dense(output_dim, kernel_regularizer=l2(1e-4)))

    # Compile
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss='mse',
        metrics=['mae']
    )
    
    return model """

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, LayerNormalization
from tensorflow.keras.optimizers import Adam

def build_rnn_model(input_shape, output_dim):
    model = Sequential()

    # --- Recurrent encoder ---
    # Small, stable GRU for smooth motion
    model.add(GRU(
        units=32,
        return_sequences=False,
        activation="tanh",
        recurrent_activation="sigmoid",
        input_shape=input_shape
    ))

    # --- Lightweight feature mixing ---
    model.add(LayerNormalization())
    model.add(Dense(32, activation="relu"))

    # --- Output ---
    model.add(Dense(output_dim))

    model.compile(
        optimizer=Adam(learning_rate=5e-4),
        loss="mse",
        metrics=["mae"]
    )

    return model


In [ ]:
n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_targets = y_train.shape[1]

model = build_rnn_model(
    input_shape=(n_timesteps, n_features),
    output_dim=n_targets
)

model.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
import numpy as np

def autoregressive_predict(model, initial_window, steps):
    """
    model: trained model with .predict()
    initial_window: np.array shape (10, 5)
    steps: how many future points to generate
    """
    window = initial_window.copy()
    outputs = []

    for _ in range(steps):
        # Model expects shape (1, 10, 5) or (10,5) depending on architecture
        pred = model.predict(window[np.newaxis, ...])  # shape (1,5)
        outputs.append(pred.squeeze())
        pred = scaler_y.inverse_transform(pred)
        pred = scaler_X.transform(pred)
        pred = pred.squeeze()  # shape (5,)

        #outputs.append(pred)

        # Slide the window: drop first row, append prediction
        window = np.vstack([window[1:], pred])

    return np.array(outputs)  # shape = (steps, 5)


In [ ]:
import matplotlib.pyplot as plt

def plot_xy_trajectory(original, preds):
    """
    original: (10, 5)  → use [:,0] = X and [:,1] = Y
    preds:    (N, 5)  → use [:,0] = X and [:,1] = Y
    """
    ox, oy = original[:, 0], original[:, 1]
    px, py = preds[:, 0], preds[:, 1]

    plt.figure(figsize=(8, 8))

    # Original trajectory
    plt.plot(ox, oy, marker='o', label="Original (input window)", linewidth=2)

    # Predicted trajectory (autoregressive)
    plt.plot(px, py, marker='x', label="Predicted (autoregressive)", linestyle='--', linewidth=2)

    # Mark start & end points
    plt.scatter(ox[0], oy[0], c='green', s=80, label="Start")
    plt.scatter(px[-1], py[-1], c='red', s=80, label="Final prediction")

    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("XY Trajectory: Original Window vs Autoregressive Predictions")
    plt.legend()
    plt.grid(True)
    plt.axis("equal")  # keep aspect ratio consistent
    plt.show()


In [ ]:
pred = model.predict(X_train[0][np.newaxis, ...])

In [ ]:
for i in range(5):
    print(X_train[0][i])
print("~~~~~~~")
print(X_train[0])
print(pred)




In [ ]:
future_steps = 10
preds = autoregressive_predict(model, X_test[0], future_steps)
print(preds.shape)   # (50, 5)

plot_xy_trajectory(X_test[0], preds)


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss Curve')
plt.legend()
plt.grid(True)
plt.show()

# Plot training & validation MAE
plt.figure(figsize=(8,5))
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Model MAE Curve')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
y_pred_norm = model.predict(X_test)

y_pred = scaler_y.inverse_transform(y_pred_norm)
y_true = scaler_y.inverse_transform(y_test)
X_true = scaler_X.inverse_transform(X_test[0])
#print(X_true)
#print(y_true[0])
print(y_pred[0])

true = np.vstack([X_true, y_true[0].reshape(1,-1)])
print("AAAAAAAAAAAAAA")
print(true)

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(true[:,1], true[:,0], label="Actual", linewidth=2, marker="o")
plt.scatter(y_pred[0][1], y_pred[0][0], label="Predicted", linestyle="--", linewidth=2, color='red')

# Styling
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Actual vs Predicted Trajectory")
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.show()